#### Import Julia packages

In [23]:
# import Pkg
# Pkg.add("JSON3")

In [24]:
using Gridap
using GridapMakie, CairoMakie, FileIO
using Gridap.FESpaces
using Gridap.ReferenceFEs
using Gridap.Arrays
using Gridap.Algebra
using Gridap.Geometry
using Gridap.Fields
using Gridap.CellData
using FillArrays
using Test
using InteractiveUtils
using PyPlot
using JSON3

In [25]:
function mark_nodes(f,model::DiscreteModel)
  topo   = get_grid_topology(model)
  coords = get_vertex_coordinates(topo)
  mask = map(f,coords)
  return mask
end
function update_labels!(e::Integer,model::CartesianDiscreteModel,f_Γ::Function,name::String)
    mask = mark_nodes(f_Γ,model)
    _update_labels_locally!(e,model,mask,name)
    nothing
end
function _update_labels_locally!(e,model::CartesianDiscreteModel{2},mask,name)
  topo   = get_grid_topology(model)
  labels = get_face_labeling(model)
  cell_to_entity = labels.d_to_dface_to_entity[end]
  entity = maximum(cell_to_entity) + e
  # Vertices
  vtxs_Γ = findall(mask)
  vtx_edge_connectivity = Array(get_faces(topo,0,1)[vtxs_Γ])
  # Edges
  edge_entries = [findall(x->any(x .∈  vtx_edge_connectivity[1:end.!=j]),
    vtx_edge_connectivity[j]) for j = 1:length(vtx_edge_connectivity)]
  edge_Γ = unique(reduce(vcat,getindex.(vtx_edge_connectivity,edge_entries),init=[]))
  labels.d_to_dface_to_entity[1][vtxs_Γ] .= entity
  labels.d_to_dface_to_entity[2][edge_Γ] .= entity
  add_tag!(labels,name,[entity])
  return cell_to_entity
end

_update_labels_locally! (generic function with 1 method)

#### Import Custom functions

In [26]:
#include("Julia_functions/solution_animation.jl")
include("Julia_functions/indicator_chi.jl")
include("Julia_functions/cost_fun.jl")
include("Julia_functions/gradient_descent.jl")
include("Julia_functions/solvers.jl")
include("Julia_functions/find.jl")
include("Julia_functions/res_plot.jl")
include("Julia_functions/get_domain.jl")

get_room (generic function with 1 method)

#### Read JSON File

In [27]:
width, height, windows, doors = get_room("Boundry.json")

(5, 3, Any[(121.20000076293945, 217.20000076293945, "Top"), (363.20000076293945, 470.20000076293945, "Top")], Any[(168.1999969482422, 262.1999969482422, "Left")])

#### Boundary conditions

In [28]:
function generate_f_Γ(bound, width, height)
    return function (x)
        any(
            (b[3] == "Top"    && (b[1] ≤ x[1] ≤ b[2]) && (x[2] ≈ height)) ||
            (b[3] == "Bottom" && (b[1] ≤ x[1] ≤ b[2]) && (x[2] ≈ 0)) ||
            (b[3] == "Left"   && (b[1] ≤ x[2] ≤ b[2]) && (x[1] ≈ 0)) ||
            (b[3] == "Right"  && (b[1] ≤ x[2] ≤ b[2]) && (x[1] ≈ width))
            for b in bound
        )
    end
end

generate_f_Γ (generic function with 1 method)

In [29]:
f_Γ_w = generate_f_Γ(windows, width, height)
f_Γ_d = generate_f_Γ(doors, width, height)
f_Γ_wall(x) = !(f_Γ_w(x) || f_Γ_d(x))

f_Γ_wall (generic function with 1 method)

#### Domain & Discretization

In [30]:
domain = (0, width , 0, height)
partition = (20,20)
model = CartesianDiscreteModel(domain,partition) 
update_labels!(1, model, f_Γ_w, "window")
update_labels!(2, model, f_Γ_d, "door")
update_labels!(3, model, f_Γ_wall, "wall")

order = 1
degree = 2*order

Ω = Triangulation(model)
dΩ = Measure(Ω, degree)

Γ_w = BoundaryTriangulation(model, tags="window")
dΓ_w = Measure(Γ_w, degree)

Γ_d = BoundaryTriangulation(model, tags="door")
dΓ_d = Measure(Γ_d, degree)

Γ_wall = BoundaryTriangulation(model, tags="wall")
dΓ_wall = Measure(Γ_wall, degree)

reffe = ReferenceFE(lagrangian,Float64,order)
Testspace = TestFESpace(model,reffe,conformity=:H1)                         
Trialspace = TransientTrialFESpace(Testspace)                                
Uspace = FESpace(model, reffe, conformity=:H1)

# fig = CairoMakie.plot(Ω)
# scatter!(Ω, marker=:circle, markersize=20, color=:blue) 
# display(fig)

UnconstrainedFESpace()

#### Time parameters

In [31]:
t0 = 0.0    # Start time
tF = 10.0   # End time
Δt = 0.05   # Timestep

0.05

#### Room parameters

In [32]:
h_wall(x) = 1.0
h_window(x) = 1.0
h_door(x) = 1.0

ρ(x)=1.225
c(x)=1020.0
k(x)=15.0

h = (h_wall, h_window, h_door) 
constants = (c, ρ, k, h) 

Toutdoor(x,t)=5.0
Tout(t)=x->Toutdoor(x,t)

price(t)=1.0

Tini(x)=20.0

TIni=interpolate_everywhere(Tini, Uspace(t0))
Tfin=interpolate_everywhere(20.0, Uspace(tF))

SingleFieldFEFunction():
 num_cells: 400
 DomainStyle: ReferenceDomain()
 Triangulation: BodyFittedTriangulation()
 Triangulation id: 10353107820422980377

#### Controll parameter

In [33]:
q_pos(x) = χ(x[1], -0.9, -0.7) * χ(x[2], 0, 0.20) + χ(x[1], 0.2, 0.4) * χ(x[2], -0.5, 0)
Q(x,t)=χ(t,0.0,3.0)*100.0*q_pos(x)
Qt(t)=x->Q(x,t)

Qt (generic function with 1 method)

#### Solver parameters

In [34]:
L2norm(u)=√((tF-t0)*∑(Δt*∑(∫(uu⋅uu)*dΩ) for (t,uu) in u))
L2skp(u)=(tF-t0)*∑(Δt*∑(∫(uu⋅uu)*dΩ) for (t,uu) in u)

γ = 1000.0

Proj(a,b,z) = min(max(a,z),b)
# Proj(z) = map(x->Proj(a,b,x),z)
a=0.0
b=1000.0
Proj(z) = [(t,FEFunction(Uspace,map(x->Proj(a,b,x), get_free_dof_values(zz)))) for (t,zz) in z]
# [(t,interpolate_everywhere(map(x->P(a,b,x),get_free_dof_values(zz)),Uspace(t))) for (t,zz) in z]


Proj (generic function with 2 methods)

#### SE Test

In [35]:
dΓ = dΓ_w, dΓ_d, dΓ_wall

ls = LUSolver()
θ = 0.5
solver = ThetaMethod(ls, Δt, θ)

solution, _, _ = SEsolver(solver, Qt, Trialspace, Testspace; dΩ, dΓ, Tout, constants) 


(Tuple{Float64, SingleFieldFEFunction{GenericCellField{ReferenceDomain}}}[(0.0, SingleFieldFEFunction()), (0.05, SingleFieldFEFunction()), (0.1, SingleFieldFEFunction()), (0.15000000000000002, SingleFieldFEFunction()), (0.2, SingleFieldFEFunction()), (0.25, SingleFieldFEFunction()), (0.3, SingleFieldFEFunction()), (0.35, SingleFieldFEFunction()), (0.39999999999999997, SingleFieldFEFunction()), (0.44999999999999996, SingleFieldFEFunction())  …  (9.55, SingleFieldFEFunction()), (9.600000000000001, SingleFieldFEFunction()), (9.650000000000002, SingleFieldFEFunction()), (9.700000000000003, SingleFieldFEFunction()), (9.750000000000004, SingleFieldFEFunction()), (9.800000000000004, SingleFieldFEFunction()), (9.850000000000005, SingleFieldFEFunction()), (9.900000000000006, SingleFieldFEFunction()), (9.950000000000006, SingleFieldFEFunction()), (10.000000000000007, SingleFieldFEFunction())], 0.0, 0.0)

#### Test Gradient DEs

In [36]:
# (Ts,qs,Ws,costs) = GradientDescent(;solveSE=SEsolver, 
# solveAE=AEsolver, 
# spaces=(Trialspace, Testspace, Uspace), 
# dΩ=dΩ, 
# dΓ=(dΓ_w,dΓ_d,dΓ_wall), 
# Q=Qt, 
# J=E, 
# ∇f=∇e, 
# P=Proj, 
# s_min=s_min,
# sminargs=nothing, 
# saveall=true, 
# tol=1e-5, 
# iter_max=2,
# armijoparas=(ρ=1/2, α_0=1, α_min=1/2^20, σ=1e-4), 
# Δt=Δt, 
# t0=t0,
# tF=tF,
# Tout=Tout,
# constants=constants,
# Tfin=Tfin)

In [37]:
# Opprett en mappe for midlertidige filer hvis den ikke eksisterer
if !isdir("tmp")
    mkdir("tmp")
end

# Initialtilstanden og resten av tidsløsningene
uh0 = solution[1][2]  # Første element er (t0, T0), vi henter T0
uh  = solution[2:end]  # De resterende løsningene

# Lagre resultater i Paraview-format (VTK)
createpvd("results") do pvd
    # Første løsning (t=0)
    pvd[0] = createvtk(Ω, "tmp/results_0.vtu", cellfields=["u" => uh0])

    # Lagrer løsninger for hver tidssteg
    for (tn, uhn) in uh
        pvd[tn] = createvtk(Ω, "tmp/results_$tn.vtu", cellfields=["u" => uhn])
    end
end


202-element Vector{String}:
 "results.pvd"
 "tmp/results_0.vtu"
 "tmp/results_0.05.vtu"
 "tmp/results_0.1.vtu"
 "tmp/results_0.15000000000000002.vtu"
 "tmp/results_0.2.vtu"
 "tmp/results_0.25.vtu"
 "tmp/results_0.3.vtu"
 "tmp/results_0.35.vtu"
 "tmp/results_0.39999999999999997.vtu"
 "tmp/results_0.44999999999999996.vtu"
 "tmp/results_0.49999999999999994.vtu"
 "tmp/results_0.5499999999999999.vtu"
 ⋮
 "tmp/results_9.45.vtu"
 "tmp/results_9.5.vtu"
 "tmp/results_9.55.vtu"
 "tmp/results_9.600000000000001.vtu"
 "tmp/results_9.650000000000002.vtu"
 "tmp/results_9.700000000000003.vtu"
 "tmp/results_9.750000000000004.vtu"
 "tmp/results_9.800000000000004.vtu"
 "tmp/results_9.850000000000005.vtu"
 "tmp/results_9.900000000000006.vtu"
 "tmp/results_9.950000000000006.vtu"
 "tmp/results_10.000000000000007.vtu"

In [38]:
#TODO: lag plot; pris, pådrag, temperatur 